## Configuración

In [1]:
from bs4 import BeautifulSoup
from requests import get

from tqdm.notebook import tqdm


year, month = 2023, 8
url = f'https://www.marbella.es/agenda/eventospormes/{year}/{month}.html'

### Peticiones web

In [2]:
response = get(url)
response

<Response [200]>

In [3]:
text = response.text

def preview_text(large_text: str) -> None:
    print(f'{len(large_text) = }')
    print(f'{large_text[:300]}\n\n...\n\n{large_text[-300:]}')

preview_text(text)

len(large_text) = 54317
<!DOCTYPE HTML>
<html prefix="og: http://ogp.me/ns#" lang="es-es" dir="ltr">
    <head>
        <meta charset="utf-8">
        <base href="https://www.marbella.es/agenda/crawler.listevents/-.html" />
	<meta http-equiv="content-type" content="text/html; charset=utf-8" />
	<meta name="keywords" conten

...

n-step-forward"></i>		</a>
	</li>
	<li>
		<a class="hasTooltip"  title="Final"  href="/agenda/crawler.listevents/-.html?start=6900">
			<i class="icon-forward"></i>		</a>
	</li>
		</ul>
	
			<input type="hidden" name="limitstart"
		       value="0"/>
	
</div>
		</form>
	</div>
	
    </body>
</html>



Podemos notar que hay referencias al endpoint `https://www.marbella.es/agenda/crawler.listevents/-.html?start=6900`, pero en este ejercicio lo vamos a ignorar.

Es posible que una request como la anterior no nos devuelva el mismo resultado que recibimos con una navegación normal. En ese caso, podemos intentar añadir headers que indiquen el navegador web que (teóricamente) estamos utilizando.

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/109.0'}

response = get(url, headers=headers)
response

<Response [200]>

In [5]:
text = response.text

preview_text(text)

len(large_text) = 436060
<!DOCTYPE html>
<html prefix="og: http://ogp.me/ns#" lang="es-es" dir="ltr">
    <head>
        <meta name="viewport" content="width=device-width, initial-scale=1">
        <link rel="icon" href="/images/android-chrome-192x192.png" sizes="any">
                <link rel="icon" href="/images/pro/escu

...

animation-slide-bottom"><a href="#" title="Volver al principio" uk-totop uk-scroll aria-label="Back to top"></a></div>




</div></div>
                                </div>
                
            
        
        </div>
    	
	
</div>

        </div>

        
        

    </body>
</html>



Para procesar el html recibido utilizamos BeautifulSoup

In [6]:
soup = BeautifulSoup(response.text)
# soup

Vamos a obtener todos los links que aparecen en la página

In [7]:
all_hrefs = [a['href'] for a in soup.find_all(href=True)]
len(all_hrefs)

449

Y filtramos por links que apunten a eventos

In [8]:
event_links = list({e for e in all_hrefs if 'eventodetalle' in e})
len(event_links)

115

In [9]:
event_links[-5:]

['/agenda/eventodetalle/34822/destino-argentina-borges-y-chale-mujeres-artistas-en-la-vanguardia.html',
 '/agenda/eventodetalle/47810/abstraccion-americana-carlos-merida.html',
 '/agenda/eventodetalle/51107/basilica-paleocristiana-de-vega-del-mar.html',
 '/agenda/eventodetalle/33524/surrealismos-de-giorgio-de-chirico-a-francis-bacon.html',
 '/agenda/eventodetalle/51959/villa-romana-de-rio-verde.html']

Exploramos lo que aparece en una página de evento

In [10]:
base_url = f'https://www.marbella.es'

event = event_links[0]

html_e = get(f'{base_url}{event}', headers=headers)

soup = BeautifulSoup(html_e.text)
# soup

Una función para obtener la información de un evento a partir de su link:

In [20]:
from time import sleep


def get_event_info(event_link: str) -> dict:

    html = get(f'{base_url}{event_link}', headers=headers)
    text = html.text
    soup = BeautifulSoup(text)
    base = {'event_link': event_link}
    try:
        div_info = soup.find('div', string='Repetición Anterior').parent.parent.parent
    except:
        return base
    info = {
        'title': div_info.h1.text,
        'category': div_info.h2.text,
        'datetime': div_info.div.div.text.replace('\xa0', '').replace('\n', ''),
        'views': int(div_info.find(attrs={'class': 'hitslabel'}).parent.text.split(':')[-1].strip())
    }
    gmaps_dict = text.split('var gmapConf = {')[-1].split('}')[0]
    try:
        place = {
            'longitude': float(gmaps_dict.split('longitude:\t\'')[-1].split("'")[0]),
            'latitude': float(gmaps_dict.split('latitude:\t\'')[-1].split("'")[0]),
            'place_name': text.split('myEventDetailMapload(')[-1].split('.html?tmpl=component')[0].split(' \"/agenda/')[-1]
        }
    except:
        place = {}
    return {**base, **info, **place}


Recorremos las urls de los eventos que hemos obtenido obteniendo su información

In [9]:
events_info = []
for event_link in tqdm(event_links):
    events_info.append(get_event_info(event_link))
    sleep(.5)

len(events_info)

  0%|          | 0/69 [00:00<?, ?it/s]

69

Podemos poner la respuesta como una tabla

In [10]:
import pandas as pd


df = pd.DataFrame(events_info)
df

,event_link,title,category,datetime,views,longitude,latitude,place_name
0,/agenda/eventodetalle/34823/destino-argentina-...,"Destino Argentina. Borges y Chale, mujeres a...","Exposiciones, Cultura y Enseñanza, Residentes ...","Miércoles, 02 Agosto 2023,10:00-15:00",202765,-4.940957,36.498058,lugares/detail/57/0/museo-ralli
1,/agenda/eventodetalle/33518/surrealismos-de-gi...,Surrealismos. De Giorgio De Chirico a Francis ...,"Exposiciones, Cultura y Enseñanza, Residentes ...","Martes, 22 Agosto 2023,10:00-15:00",272237,-4.940957,36.498058,lugares/detail/57/0/museo-ralli
2,/agenda/eventodetalle/34840/destino-argentina-...,"Destino Argentina. Borges y Chale, mujeres a...","Exposiciones, Cultura y Enseñanza, Residentes ...","Viernes, 25 Agosto 2023,10:00-15:00",202766,-4.940957,36.498058,lugares/detail/57/0/museo-ralli
3,/agenda/eventodetalle/47813/abstraccion-americ...,"Abstracción Americana, Carlos Mérida","Exposiciones, Cultura y Enseñanza","Miércoles, 16 Agosto 2023,10:00-15:00",983,-4.940957,36.498058,lugares/detail/57/0/museo-ralli
4,/agenda/eventodetalle/34835/destino-argentina-...,"Destino Argentina. Borges y Chale, mujeres a...","Exposiciones, Cultura y Enseñanza, Residentes ...","Viernes, 18 Agosto 2023,10:00-15:00",202767,-4.940957,36.498058,lugares/detail/57/0/museo-ralli
...,...,...,...,...,...,...,...,...
64,/agenda/eventodetalle/34844/destino-argentina-...,"Destino Argentina. Borges y Chale, mujeres a...","Exposiciones, Cultura y Enseñanza, Residentes ...","Jueves, 31 Agosto 2023,10:00-15:00",202785,-4.940957,36.498058,lugares/detail/57/0/museo-ralli
65,/agenda/eventodetalle/47806/abstraccion-americ...,"Abstracción Americana, Carlos Mérida","Exposiciones, Cultura y Enseñanza","Sábado, 05 Agosto 2023,10:00-15:00",1004,-4.940957,36.498058,lugares/detail/57/0/museo-ralli
66,/agenda/eventodetalle/34837/destino-argentina-...,"Destino Argentina. Borges y Chale, mujeres a...","Exposiciones, Cultura y Enseñanza, Residentes ...","Martes, 22 Agosto 2023,10:00-15:00",202786,-4.940957,36.498058,lugares/detail/57/0/museo-ralli
67,/agenda/eventodetalle/34828/destino-argentina-...,"Destino Argentina. Borges y Chale, mujeres a...","Exposiciones, Cultura y Enseñanza, Residentes ...","Miércoles, 09 Agosto 2023,10:00-15:00",202787,-4.940957,36.498058,lugares/detail/57/0/museo-ralli


Función para descargar la información de todos los eventos de un mes

In [11]:
def get_all_month_events(year: int, month: int):

    month = str(month).rjust(2, '0')
    url = f'https://www.marbella.es/agenda/eventospormes/{year}/{month}.html'
    html = get(url, headers=headers)
    soup = BeautifulSoup(html.text)
    all_hrefs = [a['href'] for a in soup.find_all(href=True)]
    event_links = list({e for e in all_hrefs if 'eventodetalle' in e})

    events_info = []
    for event_link in tqdm(event_links):
        events_info.append(get_event_info(event_link))
        sleep(.5)
    df = pd.DataFrame(events_info)
    return df

df = get_all_month_events(2023, 6)
df

  0%|          | 0/73 [00:00<?, ?it/s]

,event_link,title,category,datetime,views,longitude,latitude,place_name
0,/agenda/eventodetalle/34797/destino-argentina-...,"Destino Argentina. Borges y Chale, mujeres a...","Exposiciones, Cultura y Enseñanza, Residentes ...","Martes, 27 Junio 2023,10:00-15:00",202788,-4.940957,36.498058,lugares/detail/57/0/museo-ralli
1,/agenda/eventodetalle/47763/abstraccion-americ...,"Abstracción Americana, Carlos Mérida","Exposiciones, Cultura y Enseñanza","Miércoles, 07 Junio 2023,10:00-15:00",1007,-4.940957,36.498058,lugares/detail/57/0/museo-ralli
2,/agenda/eventodetalle/34779/destino-argentina-...,"Destino Argentina. Borges y Chale, mujeres a...","Exposiciones, Cultura y Enseñanza, Residentes ...","Jueves, 01 Junio 2023,10:00-15:00",202789,-4.940957,36.498058,lugares/detail/57/0/museo-ralli
3,/agenda/eventodetalle/37439/ciencia-y-arte-con...,Ciencia y arte: conexiones entre dos modos de ...,"Cursos y Talleres, Aula de Mayores, Cultura y ...","Martes, 06 Junio 2023,17:00-18:30",152,-4.884094,36.509178,lugares/detail/17/0/centro-cultural-hospital-r...
4,/agenda/eventodetalle/33476/surrealismos-de-gi...,Surrealismos. De Giorgio De Chirico a Francis ...,"Exposiciones, Cultura y Enseñanza, Residentes ...","Viernes, 23 Junio 2023,10:00-15:00",272262,-4.940957,36.498058,lugares/detail/57/0/museo-ralli
...,...,...,...,...,...,...,...,...
68,/agenda/eventodetalle/33461/surrealismos-de-gi...,Surrealismos. De Giorgio De Chirico a Francis ...,"Exposiciones, Cultura y Enseñanza, Residentes ...","Viernes, 02 Junio 2023,10:00-15:00",272281,-4.940957,36.498058,lugares/detail/57/0/museo-ralli
69,/agenda/eventodetalle/47770/abstraccion-americ...,"Abstracción Americana, Carlos Mérida","Exposiciones, Cultura y Enseñanza","Viernes, 16 Junio 2023,10:00-15:00",1027,-4.940957,36.498058,lugares/detail/57/0/museo-ralli
70,/agenda/eventodetalle/33474/surrealismos-de-gi...,Surrealismos. De Giorgio De Chirico a Francis ...,"Exposiciones, Cultura y Enseñanza, Residentes ...","Miércoles, 21 Junio 2023,10:00-15:00",272282,-4.940957,36.498058,lugares/detail/57/0/museo-ralli
71,/agenda/eventodetalle/47764/abstraccion-americ...,"Abstracción Americana, Carlos Mérida","Exposiciones, Cultura y Enseñanza","Jueves, 08 Junio 2023,10:00-15:00",1028,-4.940957,36.498058,lugares/detail/57/0/museo-ralli


Descarga de todos los eventos de 2023 y 2024

In [21]:
from itertools import product

dfs = []
for year, month in tqdm(product(range(2023, 2025), range(1, 13))):
    dfs.append(get_all_month_events(year, month))

df = pd.concat(dfs, ignore_index=True)
df

0it [00:00, ?it/s]

  0%|          | 0/186 [00:00<?, ?it/s]

  0%|          | 0/230 [00:00<?, ?it/s]

  0%|          | 0/266 [00:00<?, ?it/s]

  0%|          | 0/270 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

,event_link,title,category,datetime,views,longitude,latitude,place_name
0,/agenda/eventodetalle/37023/reyes-magos-en-nue...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/agenda/eventodetalle/33075/arte-contemporaneo...,"Arte contemporáneo en Oaxaca. Vanguardia, mito...","Exposiciones, Cultura y Enseñanza","Jueves, 12 Enero 2023,10:00-15:00",187973.0,-4.940957,36.498058,lugares/detail/57/0/museo-ralli
2,/agenda/eventodetalle/33088/arte-contemporaneo...,"Arte contemporáneo en Oaxaca. Vanguardia, mito...","Exposiciones, Cultura y Enseñanza","Martes, 31 Enero 2023,10:00-15:00",187974.0,-4.940957,36.498058,lugares/detail/57/0/museo-ralli
3,/agenda/eventodetalle/36845/concurso-cartel-an...,Concurso Cartel Anunciador Carnaval Marbella 2023,"Concursos, Fiestas","Miércoles, 04 Enero 2023",22303.0,NaN,NaN,NaN
4,/agenda/eventodetalle/37003/parque-magico-de-n...,Parque Mágico de Navidad,"Fiestas, Ferias y Verbenas, Fiestas","Domingo, 01 Enero 2023,11:00",8596.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1848,/agenda/eventodetalle/48120/abstraccion-americ...,"Abstracción Americana, Carlos Mérida","Exposiciones, Cultura y Enseñanza","Viernes, 18 Octubre 2024,10:00-15:00",1426.0,-4.940957,36.498058,lugares/detail/57/0/museo-ralli
1849,/agenda/eventodetalle/48126/abstraccion-americ...,"Abstracción Americana, Carlos Mérida","Exposiciones, Cultura y Enseñanza","Sábado, 26 Octubre 2024,10:00-15:00",1427.0,-4.940957,36.498058,lugares/detail/57/0/museo-ralli
1850,/agenda/eventodetalle/48109/abstraccion-americ...,"Abstracción Americana, Carlos Mérida","Exposiciones, Cultura y Enseñanza","Jueves, 03 Octubre 2024,10:00-15:00",1428.0,-4.940957,36.498058,lugares/detail/57/0/museo-ralli
1851,/agenda/eventodetalle/48119/abstraccion-americ...,"Abstracción Americana, Carlos Mérida","Exposiciones, Cultura y Enseñanza","Jueves, 17 Octubre 2024,10:00-15:00",1429.0,-4.940957,36.498058,lugares/detail/57/0/museo-ralli


In [22]:
df.to_csv('marbella_events_2023_2024.csv')